In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [ ]:
df_bis = pd.read_csv("/content/drive/MyDrive/DSW_Assingment_2/data/business.csv" , names=["name"] )
df_user_biz = pd.read_csv("/content/drive/MyDrive/DSW_Assingment_2/data/user-business.csv",header=None )

# EDA 

In [ ]:
df_bis

,name
0,Pittock Mansion
1,Legal Sea Foods
2,Eurasia Sushi Bar & Seafood
3,Piece of Cake
4,Sapporo Ramen
...,...
995,Birds Barbershop
996,Portobello Vegan Trattoria
997,Bar Carlo
998,Blue Moon Pizza


In [ ]:
df_user_biz.head

<bound method NDFrame.head of        0    1    2    3    4    5    6    7    8    9    ...  990  991  992  \
0        0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
1        0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
2        0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
3        0    0    0    0    0    0    0    0    0    0  ...    0    0    1   
4        0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
14392    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
14393    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
14394    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
14395    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
14396    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   

       993  994  995 

In [ ]:

df_user_biz.iloc[3].value_counts()

0    929
1     71
Name: 3, dtype: int64

In [ ]:

df_user_biz.iloc[2000].value_counts()

0    991
1      9
Name: 2000, dtype: int64

In [ ]:


R = df_user_biz.copy()
R_first_100 = R.iloc[:, :100]

alex_ratings = R.iloc[3]
alex_ratings_excluding_100 = alex_ratings.iloc[100:]
others_ratings_excluding_100 = R.drop(3, axis=0).iloc[:, 100:]

cos_similarities = cosine_similarity([alex_ratings_excluding_100], others_ratings_excluding_100)
cos_similarities = pd.Series(cos_similarities.flatten(), index=others_ratings_excluding_100.index)

# Compute rAlex,b for all businesses in S
rAlex = R_first_100.apply(lambda col: np.sum(col * cos_similarities), axis=0)

# Find the five businesses with the highest similarity scores
top_5_business_indices = rAlex.nlargest(5).index






In [ ]:
top_5_business_scores = rAlex[top_5_business_indices].tolist()

In [ ]:

top_5_business_names =[]


for i in top_5_business_indices:
  top_5_business_names.append(df_bis.iloc[i]["name"])


for i in range(len(top_5_business_names)):
  print(f"{top_5_business_names[i]} with score of {top_5_business_scores[i]}")


Papi's Cuban & Caribbean Grill with score of 43.039526922656655
Seven Lamps with score of 33.59818777689091
Loca Luna with score of 33.26322496381452
Farm Burger with score of 32.78294005958858
Piece of Cake with score of 12.626244095828584


## PART 2

In [ ]:


R_no_alex = R.drop(3, axis=0)
item_cos_similarities = cosine_similarity(R_no_alex.T)

alex_ratings = R.iloc[3]
rAlex = np.dot(item_cos_similarities, alex_ratings)[:100]


top_5_business_indices = rAlex.argsort()[-5:][::-1]
top_5_business_scores = rAlex[top_5_business_indices]




In [ ]:

top_5_business_names =[]


for i in top_5_business_indices:
  top_5_business_names.append(df_bis.iloc[i]["name"])


for i in range(len(top_5_business_names)):
  print(f"{top_5_business_names[i]} with score of {top_5_business_scores[i]}")

Papi's Cuban & Caribbean Grill with score of 6.810936608775577
Farm Burger with score of 6.558814913234692
Seven Lamps with score of 6.440366746451209
Loca Luna with score of 5.852681031550378
Piece of Cake with score of 3.730177864424377


## Part 3

In [ ]:
U, sigma, Vt = svds(R.to_numpy(dtype=np.float64), k=10)


sigma = np.diag(sigma)

R_estimated = np.dot(np.dot(U, sigma), Vt)


alex_ratings_estimated = R_estimated[3, :100]
top_5_business_indices = alex_ratings_estimated.argsort()[-5:][::-1]
top_5_business_scores = alex_ratings_estimated[top_5_business_indices]

In [ ]:

top_5_business_names =[]


for i in top_5_business_indices:
  top_5_business_names.append(df_bis.iloc[i]["name"])


for i in range(len(top_5_business_names)):
  print(f"{top_5_business_names[i]} with score of {top_5_business_scores[i]}")

Papi's Cuban & Caribbean Grill with score of 1.1905064199911026
Loca Luna with score of 0.876254570830219
Farm Burger with score of 0.8578263876279594
Seven Lamps with score of 0.8179473119616179
Piece of Cake with score of 0.299354393760916
